# ロジスティック回帰（実データ）【演習用】
- 実際のデータを使ってロジスティック回帰を行う

## 1. ライブラリの読み込み

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 勾配降下法（SGD）を使う分類モデル（ロジスティック回帰など）
from sklearn.linear_model import SGDClassifier
# 分類モデルの評価指標
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
# 標準化を行うためのクラス
from sklearn.preprocessing import StandardScaler

## 2. データの読み込み 
- データセット：[House Sales in King County, USA](https://www.kaggle.com/datasets/harlfoxem/housesalesprediction)
    - 20個の説明変数をもつ住宅価格のデータ
    - 今回は以下の7つの変数を使用
- [説明変数の意味](https://www.kaggle.com/code/hnnhytc/data-analysis-with-python)
    - sqft_living:	延床面積 [平方フィート]
    - sqft_lot:	敷地面積 [平方フィート]
    - sqft_above:	地下室を除いた延床面積 [平方フィート]
    - yr_built:	建築年
    - **yr_renovated:	改装された年**
    - sqft_living15:	2015年における延床面積
    - sqft_lot15:	2015年における敷地面積

In [4]:
# CSVファイルの読み込み
df_house = pd.read_csv("../../1_data/ch2/kc_house_data.csv")[
    ['yr_renovated', 'sqft_living', 'sqft_lot', 'sqft_above', 'yr_built', 'sqft_living15', 'sqft_lot15']
]
# yr_renovatedを「改装済み」を表すカテゴリ変数に変換
df_house['yr_renovated'] = df_house['yr_renovated'] > 0 

# 先程と似た中古住宅のデータ
df_house.head()

,yr_renovated,sqft_living,sqft_lot,sqft_above,yr_built,sqft_living15,sqft_lot15
0,False,1180,5650,1180,1955,1340,5650
1,True,2570,7242,2170,1951,1690,7639
2,False,770,10000,770,1933,2720,8062
3,False,1960,5000,1050,1965,1360,5000
4,False,1680,8080,1680,1987,1800,7503


In [5]:
df_house.describe()

,sqft_living,sqft_lot,sqft_above,yr_built,sqft_living15,sqft_lot15
count,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000
mean,2079.899736,1.510697e+04,1788.390691,1971.005136,1986.552492,12768.455652
std,918.440897,4.142051e+04,828.090978,29.373411,685.391304,27304.179631
min,290.000000,5.200000e+02,290.000000,1900.000000,399.000000,651.000000
25%,1427.000000,5.040000e+03,1190.000000,1951.000000,1490.000000,5100.000000
50%,1910.000000,7.618000e+03,1560.000000,1975.000000,1840.000000,7620.000000
75%,2550.000000,1.068800e+04,2210.000000,1997.000000,2360.000000,10083.000000
max,13540.000000,1.651359e+06,9410.000000,2015.000000,6210.000000,871200.000000


## 3. データの前処理
- [StandardScaler()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) を用いて標準化（詳しくは第6章で扱う）を実施

In [6]:
# 正解データ作成
y = df_house["yr_renovated"].values
# 学習用データ作成
X = df_house.drop('yr_renovated', axis=1).values

# 各特徴量を平均0，分散1となるように変換（標準化）するためのクラス
# 今回のデータセットは特徴量の平均や分散がそれぞれで大きく異なるため、
# そのままでは確率値がうまく算出されない
std_scaler = StandardScaler() 

# 標準化した特徴量X_scaledを得る
# 以降はX_scaledをモデルの学習等に使う
X_scaled = std_scaler.fit_transform(X)

## 4. [演習] モデルの実装
- yr_renovatedを目的変数として、ロジスティック回帰モデルを学習させよう
- 学習後の重みw0, w1, ..., w6を表示しよう


In [8]:
##### 以降は自分で実装しよう #####
clf=SGDClassifier(loss="log_loss",max_iter=10000,fit_intercept=True,random_state=1234,tol=1e-3)
clf.fit(X_scaled,y)
w0 = clf.intercept_[0]
w1 = clf.coef_[0, 0]
w2 = clf.coef_[0, 1]
w3 = clf.coef_[0, 2]
w4 = clf.coef_[0, 3]
w5 = clf.coef_[0, 4]
w6 = clf.coef_[0, 5]
print("w0 = {:.3f}, w1 = {:.3f}, w2 = {:.3f}, w3 = {:.3f}".format(w0, w1, w2, w3))
print("w4 = {:.3f}, w5 = {:.3f}, w6 = {:.3f}".format(w4, w5, w6))

w0 = -3.817, w1 = 0.432, w2 = -0.041, w3 = 0.101
w4 = -1.181, w5 = 0.009, w6 = 0.012


## 5. [演習] モデルの性能を確認
- 実際に予測を実行して、対数尤度を表示してみよう
- 最後に正解率と混同行列を表示してみよう

### 5-1. 予測値の算出

In [9]:
y_est =clf.predict(X_scaled)

y_est_proba = clf.predict_proba(X)

### 5-2. 対数尤度の確認

In [12]:
# log_lossは「負の」対数尤度を計算する関数であるため、- をつけて戻している
# normalizeという引数をTrueのままにしておくと、合計の対数尤度ではなく平均の対数尤度が計算されるので注意
print('対数尤度 = {:.3f}'.format(- log_loss(y, y_est_proba, normalize=False))) 

対数尤度 = -35797.503
対数尤度 = -34305.741


### 5-3. 正解率と混同行列の確認

In [16]:
print("正解率 = {:.3f}%".format(100*accuracy_score(y,y_est)))
conf_mat=pd.DataFrame(confusion_matrix(y,y_est),
    index=["正解 = 未リフォーム","正解=リフォーム済"],
    columns=["予測 = 未リフォーム","予測=リフォーム済"])
conf_mat

正解率 = 95.725%


,予測 = 未リフォーム,予測=リフォーム済
正解 = 未リフォーム,20681,18
正解=リフォーム済,906,8
